# 🎙️ RAG por Voz — DEMO v2.1 (Preflight + Fallback)

Este cuaderno permite **preguntar por audio** y obtener **respuesta hablada** usando tu pipeline `rag_mejorado` (LlamaIndex + ChromaDB + Ollama).

**Novedades v2.1**
- Celda **preflight** que revisa e instala en el **kernel activo**: `ipywidgets`, `av`, `ffmpeg`, `torch`, `transformers`, `sentence-transformers`, `safetensors`, `ctranslate2`, `sentencepiece`, conectores de **LlamaIndex** y `chromadb`.
- **Quick fix de DLLs** para Windows: agrega rutas de `Library/bin` del entorno a la búsqueda de DLLs antes de importar `av`/`faster_whisper`.
- STT con **fallback automático**: intenta `CPU int8_float16` y si no es compatible, cae a `CPU float32` (modelo `base`).

> Sugerido: Python ≥ 3.10 (ideal 3.12) y **Ollama** ejecutándose con `llama3` o `llama3:2`.


## 1) Preflight — diagnóstico e instalación en el kernel activo
Ejecuta esta celda **primero**. Si instala algo, **reinicia el kernel** y vuelve a correr desde aquí.


In [1]:

# === PRE-FLIGHT ===
import sys, os, importlib, subprocess, platform, shutil
from pathlib import Path

print("== Preflight en kernel ==>", sys.executable)
print("SO:", platform.platform())

# Helpers

def run_pip(*pkgs):
    cmd = [sys.executable, "-m", "pip", "install", "-U"] + list(pkgs)
    print(">>", " ".join(cmd))
    return subprocess.call(cmd)

def run_conda(*pkgs):
    conda_exe = shutil.which("conda")
    if not conda_exe:
        print("(!) 'conda' no está en PATH de este kernel; saltando conda.")
        return 1
    cmd = [conda_exe, "install", "-y", "-c", "conda-forge"] + list(pkgs)
    print(">>", " ".join(cmd))
    return subprocess.call(cmd)

def ensure_import(mod_name, pip_pkg=None, conda_pkg=None):
    try:
        importlib.import_module(mod_name)
        print(f"[OK] {mod_name}")
        return False
    except Exception as e:
        print(f"[FALTA] {mod_name} ({type(e).__name__}: {e})")
        changed = False
        if conda_pkg:
            rc = run_conda(conda_pkg)
            changed |= (rc == 0)
            try:
                importlib.import_module(mod_name)
                print(f"[OK] {mod_name} (via conda)")
                return True
            except Exception as e2:
                print(f"[AÚN FALTA] {mod_name} tras conda: {e2}")
        if pip_pkg:
            rc = run_pip(pip_pkg)
            changed |= (rc == 0)
            try:
                importlib.import_module(mod_name)
                print(f"[OK] {mod_name} (via pip)")
                return True
            except Exception as e3:
                print(f"[AÚN FALTA] {mod_name} tras pip: {e3}")
        return changed

changed_any = False

# ipywidgets
changed_any |= ensure_import("ipywidgets", pip_pkg="ipywidgets")
changed_any |= ensure_import("jupyterlab_widgets", pip_pkg="jupyterlab_widgets")

# AV y FFmpeg
changed_any |= ensure_import("av", pip_pkg="av", conda_pkg="av")
from shutil import which as _which
ffmpeg_path = _which("ffmpeg")
if ffmpeg_path:
    print(f"[OK] ffmpeg en PATH: {ffmpeg_path}")
else:
    print("[FALTA] ffmpeg en PATH; intentando instalar con conda-forge...")
    rc = run_conda("ffmpeg")
    ffmpeg_path = _which("ffmpeg")
    if ffmpeg_path:
        print(f"[OK] ffmpeg instalado: {ffmpeg_path}")
        changed_any = True
    else:
        print("[ATENCIÓN] ffmpeg sigue no disponible. Añade manualmente la carpeta 'bin' de ffmpeg al PATH o reinstala con conda.")

# Torch CPU si falta
try:
    import torch
    print(f"[OK] torch {torch.__version__}")
except Exception:
    print("[FALTA] torch (CPU) → instalando desde índice oficial de PyTorch CPU...")
    rc = subprocess.call([sys.executable, "-m", "pip", "install", "-U",
                          "torch==2.2.2+cpu", "--index-url", "https://download.pytorch.org/whl/cpu"])
    changed_any |= (rc == 0)
    try:
        import torch
        print(f"[OK] torch {torch.__version__}")
    except Exception as e:
        print("[AÚN FALTA] torch tras instalación:", e)

# HF stack
changed_any |= ensure_import("transformers", pip_pkg="transformers")
changed_any |= ensure_import("sentence_transformers", pip_pkg="sentence-transformers")
changed_any |= ensure_import("safetensors", pip_pkg="safetensors")

# faster-whisper backend
changed_any |= ensure_import("ctranslate2", pip_pkg="ctranslate2")
changed_any |= ensure_import("sentencepiece", pip_pkg="sentencepiece")
changed_any |= ensure_import("faster_whisper", pip_pkg="faster-whisper")

# LlamaIndex split packages
changed_any |= ensure_import("llama_index", pip_pkg="llama-index==0.10.54")
changed_any |= ensure_import("llama_index.vector_stores.chroma", pip_pkg="llama-index-vector-stores-chroma")
changed_any |= ensure_import("llama_index.llms.ollama", pip_pkg="llama-index-llms-ollama")
changed_any |= ensure_import("llama_index.embeddings.huggingface", pip_pkg="llama-index-embeddings-huggingface")
changed_any |= ensure_import("chromadb", pip_pkg="chromadb==0.5.5")

print("\n== Resumen ==")
print("ffmpeg PATH:", ffmpeg_path or "(no encontrado)")
print("Cambios realizados (instalaciones):", changed_any)

if changed_any:
    print("\n⚠️ Se instalaron/actualizaron dependencias en este kernel.")
    print("➡️ Por favor, **reinicia el kernel** (Kernel → Restart) y vuelve a ejecutar desde esta celda.")
else:
    print("\n✅ Todo listo. No es necesario reiniciar el kernel.")


== Preflight en kernel ==> C:\Users\user\anaconda3\envs\PLN\python.exe
SO: Windows-10-10.0.22631-SP0
[OK] ipywidgets
[OK] jupyterlab_widgets
[OK] av
[OK] ffmpeg en PATH: C:\Users\user\anaconda3\envs\PLN\Library\bin\ffmpeg.EXE
[OK] torch 2.9.1+cpu
[OK] transformers
[OK] sentence_transformers
[OK] safetensors
[OK] ctranslate2
[OK] sentencepiece
[OK] faster_whisper
[OK] llama_index
[OK] llama_index.vector_stores.chroma
[OK] llama_index.llms.ollama
[OK] llama_index.embeddings.huggingface
[OK] chromadb

== Resumen ==
ffmpeg PATH: C:\Users\user\anaconda3\envs\PLN\Library\bin\ffmpeg.EXE
Cambios realizados (instalaciones): False

✅ Todo listo. No es necesario reiniciar el kernel.


## 2) Cargar `rag_mejorado`
- Si es **.ipynb**, usa `%run rag_mejorado.ipynb`.
- Si es **.py**, se importará como módulo.


In [6]:

# 👉 Descomenta si tu archivo es notebook:
%run RAG_LLAMA3.ipynb

# 👉 Si es .py, intenta importarlo:
try:
    import importlib
    rag = importlib.import_module('RAG_LLAMA3')
    from RAG_LLAMA3 import consultar_rag_databse_final
    print("Importado 'RAG_LLAMA3' como módulo.")
    try:
        print("Vectores en colección:", rag.chroma_collection_final.count())
    except Exception as e:
        print("No se pudo leer conteo de vectores:", e)
except Exception as e:
    print("No se pudo importar 'RAG_LLAMA3.py'. Si lo tienes como .ipynb usa: %run RAG_LLAMA3.ipynb")


>> C:\Users\user\anaconda3\envs\PLN\python.exe -m pip install -U llama-index>=0.11.20 llama-index-llms-ollama>=0.2.2 llama-index-embeddings-huggingface>=0.2.4 llama-index-vector-stores-chroma>=0.2.0 chromadb>=0.5.5 sentence-transformers>=3.0.0 pandas>=2.1.0
✅ Dependencias instaladas/actualizadas correctamente.
✅ Módulo disponible: llama_index
✅ Módulo disponible: llama_index.llms.ollama
✅ Módulo disponible: llama_index.vector_stores.chroma
✅ Módulo disponible: chromadb

✅ Preflight + Imports + Configuración completados.
Imports configurados
Metodología basada en: Rangan & Yin (2024) - RAG + Fine-tuning
CARGANDO CSV ORIGINAL

Total registros: 2,977,413
Registros relevantes: 16,724

Distribución por tipo:
tipo
REMATE               9282
JUNTA_ACCIONISTAS    3772
DISOLUCION           3670
Name: count, dtype: int64

EXTRAYENDO EMPRESAS DEL TEXTO COMPLETO

Extrayendo... (2-3 minutos)

Completado en 0.5 segundos

Resultados:
  Total: 16,724
  Extraídas: 3,011 (18.0%)
  No extraídas: 13,713

D

## 3) Verificar Ollama y configurar el modelo


In [7]:

import subprocess, re
from llama_index.core import Settings

out = subprocess.run("ollama list", shell=True, capture_output=True, text=True)
print(out.stdout or out.stderr)
model_name = None
if "llama3:2" in (out.stdout or ""):
    model_name = "llama3:2"
elif re.search(r"\bllama3\b", out.stdout or ""):
    model_name = "llama3"
print("Modelo sugerido:", model_name or "(ajústalo manualmente si es necesario)")

try:
    from llama_index.llms.ollama import Ollama
    if model_name:
        Settings.llm = Ollama(model=model_name, request_timeout=300.0,
                              system_prompt=(
                                  "Eres un asistente experto en documentos legales peruanos. "
                                  "Responde en español usando solo el contexto."
                              ))
        print("LLM configurado con:", model_name)
except Exception as e:
    print("No fue posible configurar Ollama:", e)


NAME               ID              SIZE      MODIFIED     
llama3:latest      365c0bd3c000    4.7 GB    38 hours ago    
llama3.2:latest    a80c4f17acd5    2.0 GB    8 days ago      

Modelo sugerido: llama3
LLM configurado con: llama3


## 4) Quick fix DLLs (Windows)
Si estás en Windows y PyAV/FFmpeg dan **DLL load failed**, ejecuta esta celda **antes** de importar `faster_whisper`.


In [8]:

import os, sys
base = sys.prefix
add_dirs = [
    os.path.join(base, "Library", "bin"),
    os.path.join(base, "DLLs"),
    os.path.join(base, "Scripts"),
]
for d in add_dirs:
    if os.path.isdir(d):
        try:
            os.add_dll_directory(d)
            print("Añadido al buscador de DLLs:", d)
        except Exception as e:
            print("No se pudo añadir", d, e)

# Prueba rápida
try:
    import av
    from pydub.utils import which
    print("PyAV OK:", getattr(av, '__version__', 'OK'), "| ffmpeg:", which("ffmpeg") or "(no en PATH)")
except Exception as e:
    print("PyAV aún falla:", e)


Añadido al buscador de DLLs: C:\Users\user\anaconda3\envs\PLN\Library\bin
Añadido al buscador de DLLs: C:\Users\user\anaconda3\envs\PLN\DLLs
Añadido al buscador de DLLs: C:\Users\user\anaconda3\envs\PLN\Scripts
PyAV OK: 16.1.0 | ffmpeg: C:\Users\user\anaconda3\envs\PLN\Library\bin\ffmpeg.exe


## 5) DEMO por voz (Gradio) — Fallback automático
- Transcripción con `faster-whisper` (CPU `int8_float16` → `float32` si falla).
- Consulta `consultar_rag_final(...)`.
- TTS con `pyttsx3`.


In [ ]:

import gradio as gr
import tempfile, os
from faster_whisper import WhisperModel
import pyttsx3
from pydub.utils import which

print("ffmpeg:", which("ffmpeg") or "(no encontrado)")

# --- STT con fallback ---
stt_info = ""

def build_stt_model():
    global stt_info
    try:
        m = WhisperModel("small", device="cpu", compute_type="int8_float16")
        stt_info = "STT: CPU int8_float16"
        return m
    except Exception as e:
        stt_info = f"STT: int8_float16 no disponible → usando CPU float32 (motivo: {type(e).__name__})"
        return WhisperModel("base", device="cpu", compute_type="float32")

stt_model = build_stt_model()
print(stt_info)

# --- TTS ---
tts_engine = pyttsx3.init()
tts_engine.setProperty('rate', 170)
tts_engine.setProperty('volume', 1.0)

TIPOS = ["", "DISOLUCION", "REMATE", "JUNTA_ACCIONISTAS"]

def transcribir(audio_path: str) -> str:
    segments, info = stt_model.transcribe(audio_path, language="es", beam_size=5)
    return " ".join([s.text.strip() for s in segments]).strip()

def tts_to_wav(text: str) -> str:
    tmp_wav = tempfile.mktemp(suffix=".wav")
    tts_engine.save_to_file(text, tmp_wav)
    tts_engine.runAndWait()
    return tmp_wav

def rag_voice_pipeline(audio, tipo, mes, anio, top_k):
    if audio is None:
        return "No se recibió audio.", None

    audio_path = audio  # Gradio entrega filepath

    # 1) STT
    try:
        pregunta = transcribir(audio_path)
    except Exception as e:
        return f"Error al transcribir: {type(e).__name__} - {e}", None

    # 2) Normalizar filtros
    tipo = (tipo or "").strip() or None
    mes  = (mes or "").strip() or None
    if mes and len(mes) == 1:
        mes = mes.zfill(2)
    anio = (anio or "").strip() or '2025'

    # 3) Consultar RAG
    try:
        resp, nodes, t = consultar_rag_final(pregunta, tipo=tipo, mes=mes, año=anio, top_k=int(top_k), verbose=False)
        if not resp:
            resp = "No se encontró contexto suficiente para responder con precisión."
    except Exception as e:
        resp = f"Error al consultar RAG: {type(e).__name__} - {e}"
        nodes = []

    # 4) Resumen nodos (Top 5)
    resumen = []
    for n in nodes[:5]:
        resumen.append(f"• score={getattr(n,'score',None):.3f} | tipo={n.metadata.get('tipo')} | fecha={n.metadata.get('fecha')} | mes={n.metadata.get('mes')} | año={n.metadata.get('año')}")
    resumen_md = "\n".join(resumen) if resumen else "(Sin nodos recuperados)"

    # 5) TTS
    try:
        wav_out = tts_to_wav(resp)
    except Exception:
        wav_out = None

    texto_md = (
        f"**Modo STT:** {stt_info}\n\n"
        f"**Pregunta (transcrita):** {pregunta}\n\n"
        f"**Respuesta:**\n\n{resp}\n\n"
        f"**Nodos (Top 5):**\n\n{resumen_md}"
    )
    return texto_md, wav_out

with gr.Blocks(title="RAG por Voz — v2.1 (preflight + fallback)") as demo:
    gr.Markdown("## 🎙️ Pregunta por voz y respuesta hablada")
    audio_in = gr.Audio(sources=["microphone", "upload"], type="filepath", label="Audio (micrófono o archivo)")
    with gr.Row():
        tipo_in = gr.Dropdown(choices=TIPOS, value="", label="Tipo (opcional)")
        mes_in  = gr.Textbox(value="", label="Mes (MM)")
        anio_in = gr.Textbox(value="2025", label="Año (YYYY)")
        topk_in = gr.Slider(3, 20, value=8, step=1, label="Top-K")
    btn = gr.Button("🔎 Consultar")
    texto_out = gr.Markdown()
    audio_out = gr.Audio(label="Respuesta en audio", type="filepath")

    btn.click(rag_voice_pipeline, inputs=[audio_in, tipo_in, mes_in, anio_in, topk_in], outputs=[texto_out, audio_out])

# Ejecutar la app
demo.launch(inbrowser=True, debug=True)


ffmpeg: C:\Users\user\anaconda3\envs\PLN\Library\bin\ffmpeg.exe
STT: int8_float16 no disponible → usando CPU float32 (motivo: ValueError)
* Running on local URL:  http://127.0.0.1:7865
* To create a public link, set `share=True` in `launch()`.


Exception in callback _ProactorBasePipeTransport._call_connection_lost(None)
handle: <Handle _ProactorBasePipeTransport._call_connection_lost(None)>
Traceback (most recent call last):
  File "C:\Users\user\anaconda3\envs\PLN\lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\user\anaconda3\envs\PLN\lib\asyncio\proactor_events.py", line 165, in _call_connection_lost
    self._sock.shutdown(socket.SHUT_RDWR)
ConnectionResetError: [WinError 10054] Se ha forzado la interrupción de una conexión existente por el host remoto
Exception in callback _ProactorBasePipeTransport._call_connection_lost(None)
handle: <Handle _ProactorBasePipeTransport._call_connection_lost(None)>
Traceback (most recent call last):
  File "C:\Users\user\anaconda3\envs\PLN\lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\user\anaconda3\envs\PLN\lib\asyncio\proactor_events.py", line 165, in _call_connecti